In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
# from urllib.robotparser import RobotFileParser

# rp = RobotFileParser()
# rp.set_url("https://www.film.ru/robots.txt")
# rp.read()

# url = "https://www.film.ru/a-z/serials?sort=imdb"
# print("Can fetch?", rp.can_fetch("*", url))

Can fetch? True


In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# BASE_URL = "https://myshows.me"
# PAGE_URL = "https://myshows.me/search/all/?page={}/"


# def get_shows_from_page(page=1):
#     url = PAGE_URL.format(page)
#     response = requests.get(url)
#     response.raise_for_status()

#     soup = BeautifulSoup(response.text, "html.parser")
#     shows = []

#     # Каждая карточка — это <a href="/view/...">
#     for card in soup.select("a[href^='/view/']"):
#         link = BASE_URL + card.get("href")

#         # внутри ссылки название лежит в определённом div
#         title_el = card.select_one("div > div:nth-of-type(1) > div:nth-of-type(1)")
#         if title_el:
#             title = title_el.get_text(strip=True)
#             shows.append((title, link))

#     return shows


# if __name__ == "__main__":
#     all_shows = []
#     page = 1
#     while len(all_shows) < 20:  # берём первые 20 сериалов
#         shows = get_shows_from_page(page)
#         all_shows.extend(shows)
#         page += 1

#     df = pd.DataFrame(all_shows[:20], columns=["Название", "Ссылка"])
#     print(df)
#     df.to_csv("shows_20.csv", index=False, encoding="utf-8-sig")

                       Название                          Ссылка
0                        Шерлок   https://myshows.me/view/7718/
1                Игра престолов  https://myshows.me/view/11945/
2            Сверхъестественное     https://myshows.me/view/45/
3        Теория большого взрыва      https://myshows.me/view/2/
4                 Во все тяжкие    https://myshows.me/view/187/
5                       Отбросы    https://myshows.me/view/304/
6              Остаться в живых      https://myshows.me/view/8/
7                   Доктор Хаус      https://myshows.me/view/1/
8   Американская история ужасов  https://myshows.me/view/19638/
9           Очень странные дела  https://myshows.me/view/48017/
10             Ходячие мертвецы   https://myshows.me/view/5317/
11               Черное зеркало  https://myshows.me/view/22410/
12                       Друзья     https://myshows.me/view/20/
13                  Обмани меня      https://myshows.me/view/3/
14             Дневники вампира    https

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
from time import sleep
import html
import re

BASE_URL = "https://myshows.me"
PAGE_URL = "https://myshows.me/search/all/?page={}/"

RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)


def get_page_url(page: int) -> str:
    if page == 1:
        return "https://myshows.me/search/all/"
    else:
        return f"https://myshows.me/search/all/?page={page}"


def get_shows_links(limit=20):
    """Собираем ссылки на сериалы из каталога"""
    all_shows = []
    page = 1
    while len(all_shows) < limit:
        url = get_page_url(page)
        response = requests.get(url)
        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.text, "html.parser")
        cards = soup.select("a[href^='/view/']")
        if not cards:
            break

        for card in cards:
            link = BASE_URL + card.get("href")
            title_el = card.select_one("div > div:nth-of-type(1) > div:nth-of-type(1)")
            if title_el:
                title = title_el.get_text(strip=True)
                all_shows.append((title, link))
                if len(all_shows) >= limit:
                    break
        page += 1
    return all_shows


def parse_show(show_id, title, url):
    """Парсим страницу конкретного сериала"""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # таблица с характеристиками
    info_table = soup.select_one("table")
    start_date = end_date = country = genres = ""
    episodes = duration = None
    if info_table:
        rows = info_table.select("tr")
        try:
            # даты: "2010—2017" или "2019" или "2010 — ..."
            date_text = rows[0].select_one("td:nth-of-type(2)").get_text(strip=True)
            years = re.findall(r"\d{4}", date_text)

            if len(years) >= 1:
                start_date = years[0]
            if len(years) >= 2:
                end_date = years[1]
            else:
                end_date = ""
        except:
            pass
        try:
            country = rows[1].select_one("td:nth-of-type(2)").get_text(strip=True)
        except:
            pass
        try:
            genres = rows[2].select_one("td:nth-of-type(2)").get_text(" ", strip=True)
        except:
            pass
        try:
            duration_text = rows[6].select_one("td:nth-of-type(2)").get_text(strip=True)
            duration_match = re.search(r"\d+", duration_text)
            if duration_match:
                duration = int(duration_match.group(0))
        except:
            pass
        try:
            episodes_text = rows[7].select_one("td:nth-of-type(2)").get_text(strip=True)
            episodes_match = re.search(r"\d+", episodes_text.replace("~", ""))
            if episodes_match:
                episodes = int(episodes_match.group(0))
        except:
            pass

    # описание и "в чём суть?"
    description = ""
    whats_the_point = ""
    desc_block = soup.select_one(
        "main div:nth-of-type(5) div div div div div:nth-of-type(2)"
    )
    if desc_block:
        ps = desc_block.select("p")
        if len(ps) > 0:
            description = html.unescape(ps[0].get_text(" ", strip=True))
        if len(ps) > 1:
            whats_the_point = html.unescape(ps[1].get_text(" ", strip=True))

    # постер (только src)
    poster_url = ""
    poster_el = soup.select_one("img[itemprop='contentUrl']")
    if poster_el:
        poster_url = poster_el.get("src", "")

    return {
        "id": show_id,
        "title": title,
        "url": url,
        "start_date": start_date,
        "end_date": end_date,
        "country": country,
        "genres": genres,
        "episode_duration_minutes": duration,
        "episodes": episodes,
        "description": description,
        "whats_the_point": whats_the_point,
        "poster_url": poster_url,
    }


def save_results(results, suffix=""):
    """Сохраняем промежуточные результаты"""
    df = pd.DataFrame(results)
    csv_path = os.path.join(RESULTS_DIR, f"shows{suffix}.csv")
    json_path = os.path.join(RESULTS_DIR, f"shows{suffix}.json")

    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"💾 Сохранено {len(results)} сериалов в {csv_path} и {json_path}")


def main(limit=20):
    shows = get_shows_links(limit)
    results = []

    for i, (title, link) in enumerate(shows, start=1):
        print(f"[{i}/{limit}] Парсим {title} ...")
        try:
            data = parse_show(i, title, link)
            results.append(data)
            # сохраняем каждые 100 сериалов
            if i % 100 == 0:
                save_results(results, suffix=f"_part{i}")
            sleep(0.1)
        except Exception as e:
            print(f"Ошибка при парсинге {title}: {e}")

    # финальное сохранение
    save_results(results)


if __name__ == "__main__":
    main(limit=5000)  # здесь можно менять число

[1/5000] Парсим Шерлок ...
[2/5000] Парсим Игра престолов ...
[3/5000] Парсим Сверхъестественное ...
[4/5000] Парсим Теория большого взрыва ...
[5/5000] Парсим Во все тяжкие ...
[6/5000] Парсим Отбросы ...
[7/5000] Парсим Остаться в живых ...
[8/5000] Парсим Доктор Хаус ...
[9/5000] Парсим Американская история ужасов ...
[10/5000] Парсим Очень странные дела ...
[11/5000] Парсим Ходячие мертвецы ...
[12/5000] Парсим Черное зеркало ...
[13/5000] Парсим Друзья ...
[14/5000] Парсим Обмани меня ...
[15/5000] Парсим Дневники вампира ...
[16/5000] Парсим Как я встретил вашу маму ...
[17/5000] Парсим Бесстыжие ...
[18/5000] Парсим Декстер ...
[19/5000] Парсим Настоящий детектив ...
[20/5000] Парсим Клиника ...
[21/5000] Парсим Половое воспитание ...
[22/5000] Парсим Ганнибал ...
[23/5000] Парсим Острые козырьки ...
[24/5000] Парсим Чернобыль ...
[25/5000] Парсим Игра в кальмара ...
[26/5000] Парсим Люцифер ...
[27/5000] Парсим Однажды в сказке ...
[28/5000] Парсим Гравити Фолз ...
[29/5000] Па